In [0]:
! mkdir ~/.kaggle/ && echo '{"username":"nashikunov","key":""}' > ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [0]:
import kaggle

In [0]:
kaggle.api.competition_download_files('chinese-char-recognition-smmo19')

In [0]:
! kaggle competitions download -c chinese-char-recognition-smmo19

test.npy.zip: Skipping, found more recently modified local copy (use --force to force download)
train-4.npy.zip: Skipping, found more recently modified local copy (use --force to force download)
train-1.npy.zip: Skipping, found more recently modified local copy (use --force to force download)
random_labels.csv: Skipping, found more recently modified local copy (use --force to force download)
train-3.npy.zip: Skipping, found more recently modified local copy (use --force to force download)
train-2.npy.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!ls

chinese-char-recognition-smmo19.zip  test.npy.zip     train-3.npy
random_labels.csv		     train-1.npy      train-3.npy.zip
sample_data			     train-1.npy.zip  train-4.npy
submission.csv			     train-2.npy      train-4.npy.zip
test.npy			     train-2.npy.zip


In [0]:
! unzip train-1.npy.zip
! unzip train-2.npy.zip
! unzip train-3.npy.zip
! unzip train-4.npy.zip
! unzip test.npy.zip

Archive:  train-1.npy.zip
replace train-1.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train-1.npy             
Archive:  train-2.npy.zip
replace train-2.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train-2.npy             
Archive:  train-3.npy.zip
replace train-3.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train-3.npy             
Archive:  train-4.npy.zip
replace train-4.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train-4.npy             
Archive:  test.npy.zip
replace test.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.npy                


In [0]:
!ls

chinese-char-recognition-smmo19.zip  test.npy.zip     train-3.npy
random_labels.csv		     train-1.npy      train-3.npy.zip
sample_data			     train-1.npy.zip  train-4.npy
submission.csv			     train-2.npy      train-4.npy.zip
test.npy			     train-2.npy.zip


In [0]:
import numpy as np
import tensorflow as tf

batch_size = 128
val_size = 0.1

data = np.load("train-1.npy", allow_pickle=True)

for i in range(1, 5):
  data = np.concatenate([data, np.load(f'train-{i}.npy', allow_pickle=True)])

data_test = np.load("test.npy", allow_pickle=True)

In [0]:
data_test.shape

(83247,)

In [0]:
data.shape

(416233, 2)

In [0]:
np.random.shuffle(data)

In [0]:
for el in data[:10]:
  print(el[0].shape)

(95, 94)
(104, 59)
(83, 91)
(96, 81)
(70, 69)
(80, 51)
(83, 72)
(74, 76)
(125, 83)
(75, 84)


In [0]:
char_to_id = {char:id for id, char in enumerate(np.unique([char for _, char in data]))}
id_to_char = {id:char for id, char in enumerate(np.unique([char for _, char in data]))}

def train_gen():
  for img, label in data[int((val_size * len(data))):]:
    img = img[:, :, np.newaxis]
    label = char_to_id[label]
    yield img, label

def val_gen():
  for img, label in data[:int((val_size * len(data)))]:
    img = img[:, :, np.newaxis]
    label = char_to_id[label]
    yield img, label

def test_gen():
  for img in data_test:
    img = img[:, :, np.newaxis]
    yield img

In [0]:
dataset_train = tf.data.Dataset.from_generator(
    train_gen,
    output_types=(tf.uint8, tf.int32),
    output_shapes=((None, None, 1), ())
)

dataset_val = tf.data.Dataset.from_generator(
    val_gen,
    output_types=(tf.uint8, tf.int32),
    output_shapes=((None, None, 1), ())
)

dataset_test = tf.data.Dataset.from_generator(
    test_gen,
    output_types=tf.uint8,
    output_shapes=(None, None, 1)
)

In [0]:
def augmentation(img):
  rand = np.random.rand()
  if rand <= 0.16:
    img = tf.image.random_brightness(img, 0.4)
    return img
  if rand <= 0.32: 
    img = tf.image.adjust_gamma(img, 0.4)
    return img
  if rand <= 0.48:
    img = tf.image.apply_affine_transform(img, shear=35)
    return img
  if rand <= 0.64:
    img = tf.image.apply_brightness_shift(img, 0.5)
    return img
  if rand <= 0.8:
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    return img
  if rand <= 1:
    img = tf.image.rot90(img, tf.random_uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    return img

def preprocess_train(img, label):
  img = tf.image.resize(img, (80, 80))
  img = tf.cast(img, tf.float32)
  img = (img - 127.5) / 127.5
  if np.random.rand() < 0.5:
    img = augmentation(img)
  label = tf.one_hot(label, 1000)
  return img, label

def preprocess_val(img, label):
  img = tf.image.resize(img, (80, 80))
  img = tf.cast(img, tf.float32)
  img = (img - 127.5) / 127.5
  label = tf.one_hot(label, 1000)
  return img, label

def preprocess_test(img):
  img = tf.image.resize(img, (80, 80))
  img = tf.cast(img, tf.float32)
  img = (img - 127.5) / 127.5
  return img

In [0]:
INIT_LR = 2e-4
EPOCHS = 20

In [0]:
dataset_train = dataset_train.map(preprocess_train, num_parallel_calls=-1).shuffle(1024).repeat(EPOCHS).prefetch(-1).batch(batch_size) #prefetch заранее батчи сделать скок можно

In [0]:
dataset_val = dataset_val.map(preprocess_val, num_parallel_calls=-1).shuffle(1024).repeat(EPOCHS).prefetch(-1).batch(batch_size)
dataset_test = dataset_test.map(preprocess_test, num_parallel_calls=-1).prefetch(-1).batch(batch_size)

In [0]:
initializer = tf.keras.initializers.lecun_normal()

In [0]:

def lr_scheduler(epoch):
    return INIT_LR * 0.8 ** epoch  

In [0]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_acc',
                           patience=4,
                           verbose=1,
                           min_delta=1e-4)

In [0]:

base_model = tf.keras.applications.Xception(input_shape=(80, 80, 1),
                                               include_top=True,
                                               weights=None)

In [0]:
base_model.trainable = True

model = tf.keras.Sequential([
  base_model,
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())
model.fit(dataset_train, validation_data = dataset_val, epochs=EPOCHS, callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler), 
                                                                               early_stop], steps_per_epoch=2920, validation_steps=320)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 1000)              22909904  
Total params: 22,909,904
Trainable params: 22,855,376
Non-trainable params: 54,528
_________________________________________________________________
None
Train on 2920 steps, validate on 320 steps
Epoch 1/40
2920/2920 [==============================] - 641s 219ms/step - loss: 0.6283 - acc: 0.8748 - val_loss: 0.2496 - val_acc: 0.9279
Epoch 2/40
2920/2920 [==============================] - 628s 215ms/step - loss: 0.0623 - acc: 0.9828 - val_loss: 0.0954 - val_acc: 0.9726
Epoch 3/40
2920/2920 [==============================] - 628s 215ms/step - loss: 0.0307 - acc: 0.9911 - val_loss: 0.0881 - val_acc: 0.9760
Epoch 4/40
2920/2920 [==============================] - 628s 215ms/step - loss: 0.0156 - acc: 0.9957 - val_loss: 0.0586 - val_acc: 0.9843
Epoch 5/40
2920/2920 [=========

KeyboardInterrupt: ignored

In [0]:
submission = model.predict(dataset_test)

In [0]:
submission = submission.argmax(1)
import pandas as pd

In [0]:
result = pd.read_csv('random_labels.csv')
result['Category'] = submission
result['Category'] = result['Category'].apply(lambda x: id_to_char[x])
result.head()

,Id,Category
0,1,63955
1,2,64432
2,3,64709
3,4,64177
4,5,61881


In [0]:
result.to_csv("submission.csv", index=False)

In [0]:
! kaggle competitions submit -c chinese-char-recognition-smmo19 -m 'Shikunov Nikolay IAD2' -f submission.csv

100% 965k/965k [00:00<00:00, 3.13MB/s]
Successfully submitted to Chinese character recognition